In [1]:
!pip install pyspark findspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 12.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2e002a18bb30dac87efaabf4f8b2d4fe2622dcfda5a1d881025e4b8becda0016
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import findspark
from datetime import datetime
from math import sqrt
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import operator
from itertools import starmap
findspark.init()
# sc.stop()

conf = SparkConf().setAppName("2AMD15-MS1_Part3")


sc = SparkContext(conf=conf)
ss = SparkSession(sc).builder.config("spark.driver.memory", "6g").getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
# Read files
weather = ss.read.csv('/gdrive/My Drive/aligned data/weather-ta2.csv')
stocks = ss.read.csv('/gdrive/My Drive/aligned data/stocks-ta2.csv')

In [5]:
weather = weather.drop('_c0','_c4')
weather.show()

+-----------+----------+-------+
|        _c1|       _c2|    _c3|
+-----------+----------+-------+
| station_id|      date|tmp_val|
|01016099999|2016-01-01|      4|
|01016099999|2016-01-02|      6|
|01016099999|2016-01-03|      0|
|01016099999|2016-01-04|      0|
|01016099999|2016-01-05|    -53|
|01016099999|2016-01-06|   -104|
|01016099999|2016-01-07|    -62|
|01016099999|2016-01-08|   -102|
|01016099999|2016-01-09|    -72|
|01016099999|2016-01-10|    -69|
|01016099999|2016-01-11|    -93|
|01016099999|2016-01-12|   -126|
|01016099999|2016-01-13|    -86|
|01016099999|2016-01-14|    -71|
|01016099999|2016-01-15|    -61|
|01016099999|2016-01-16|    -62|
|01016099999|2016-01-17|    -27|
|01016099999|2016-01-18|    -30|
|01016099999|2016-01-19|    -42|
+-----------+----------+-------+
only showing top 20 rows



In [6]:
stocks = stocks.drop('_c0')
stocks.show()

+--------------------+----------+-----+
|                 _c1|       _c2|  _c3|
+--------------------+----------+-----+
|          stock_name|      date|price|
|1004.Asien--Austr...|2016-01-01| 57.5|
|1004.Asien--Austr...|2016-01-02| 57.5|
|1004.Asien--Austr...|2016-01-03| 57.5|
|1004.Asien--Austr...|2016-01-04| 57.8|
|1004.Asien--Austr...|2016-01-05|57.17|
|1004.Asien--Austr...|2016-01-06|57.65|
|1004.Asien--Austr...|2016-01-07| 58.4|
|1004.Asien--Austr...|2016-01-08|56.44|
|1004.Asien--Austr...|2016-01-09|56.44|
|1004.Asien--Austr...|2016-01-10|56.44|
|1004.Asien--Austr...|2016-01-11| 56.0|
|1004.Asien--Austr...|2016-01-12|56.72|
|1004.Asien--Austr...|2016-01-13|56.11|
|1004.Asien--Austr...|2016-01-14|55.08|
|1004.Asien--Austr...|2016-01-15|56.97|
|1004.Asien--Austr...|2016-01-16|56.97|
|1004.Asien--Austr...|2016-01-17|56.97|
|1004.Asien--Austr...|2016-01-18|54.69|
|1004.Asien--Austr...|2016-01-19|56.33|
+--------------------+----------+-----+
only showing top 20 rows



In [7]:
weather = weather.rdd
stocks = stocks.rdd

In [8]:
header = weather.first()
weather = weather.filter(lambda row: row != header) # filter out the header

In [9]:
print(weather.take(10))

[Row(_c1='01016099999', _c2='2016-01-01', _c3='4'), Row(_c1='01016099999', _c2='2016-01-02', _c3='6'), Row(_c1='01016099999', _c2='2016-01-03', _c3='0'), Row(_c1='01016099999', _c2='2016-01-04', _c3='0'), Row(_c1='01016099999', _c2='2016-01-05', _c3='-53'), Row(_c1='01016099999', _c2='2016-01-06', _c3='-104'), Row(_c1='01016099999', _c2='2016-01-07', _c3='-62'), Row(_c1='01016099999', _c2='2016-01-08', _c3='-102'), Row(_c1='01016099999', _c2='2016-01-09', _c3='-72'), Row(_c1='01016099999', _c2='2016-01-10', _c3='-69')]


In [10]:
header1 = stocks.first()
stocks = stocks.filter(lambda row: row != header1) # filter out the header

In [11]:
print(stocks.take(10))

[Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-01', _c3='57.5'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-02', _c3='57.5'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-03', _c3='57.5'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-04', _c3='57.8'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-05', _c3='57.17'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-06', _c3='57.65'), Row(_c1='1004.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Domino-s-Pizza-Enterprises-Ltd_011404', _c2='2016-01-07', _c3='58.4'), Row

In [12]:
type(weather)

pyspark.rdd.PipelinedRDD

In [36]:
weather_sorted = weather.map(lambda row: (str(row[0]),(datetime.strptime(row[1],"%Y-%m-%d"),float(row[2])))) \
            .groupByKey()\
            .map(lambda row: (row[0],list(sorted(row[1],key= lambda v:v[0],reverse=False))))\
            .map(lambda row: (row[0],[v_i[1] for v_i in row[1]])).take(20)

In [14]:
weather_sorted = sc.parallelize(weather_sorted,4)

In [15]:
stocks_sorted = stocks.map(lambda row: (str(row[0]),(datetime.strptime(row[1],"%Y-%m-%d"),float(row[2])))) \
        .groupByKey()\
        .map(lambda row: (row[0],list(sorted(row[1],key= lambda v:v[0],reverse=False))))\
        .map(lambda row: (row[0],[v_i[1] for v_i in row[1]])).take(20)
# stocks_formatted = stocks.take(10)
#Only for DEBUG also remove take(3) above
stocks_sorted = sc.parallelize(stocks_sorted,4)

In [16]:
weather_sorted_filtered = weather_sorted.cartesian(weather_sorted).filter(lambda row: row[0][0] < row[1][0])

In [17]:
print(weather_sorted_filtered.take(1))

[(('01016099999', [4.0, 6.0, 0.0, 0.0, -53.0, -104.0, -62.0, -102.0, -72.0, -69.0, -93.0, -126.0, -86.0, -71.0, -61.0, -62.0, -27.0, -30.0, -42.0, -70.0, -49.0, -38.0, -28.0, -17.0, -15.0, -45.0, -55.0, -63.0, -19.0, -11.0, -46.0, -27.0, -41.0, -82.0, -54.0, -76.0, -114.0, -116.0, -148.0, -86.0, -35.0, -90.0, -136.0, -141.0, -84.0, -62.0, -20.0, -34.0, -53.0, -14.0, -25.0, -46.0, -19.0, -16.0, -21.0, -44.0, -39.0, -48.0, -66.0, -81.0, -88.0, -31.0, -9.0, -10.0, -15.0, -49.0, -44.0, -32.0, -36.0, -25.0, -76.0, -8.0, 1.0, -22.0, -38.0, -119.0, -136.0, -111.0, -94.0, -162.0, -122.0, -138.0, -180.0, -178.0, -122.0, -41.0, -85.0, -18.0, -15.0, -46.0, -41.0, -97.0, -159.0, -130.0, -91.0, -40.0, -44.0, -26.0, -38.0, -56.0, -70.0, -184.0, -91.0, -97.0, -99.0, -139.0, -86.0, -148.0, -162.0, -68.0, -81.0, -95.0, -84.0, -48.0, -60.0, -98.0, -95.0, -62.0, -57.0, -62.0, -43.0, -30.0, -39.0, -91.0, -103.0, -78.0, -53.0, -53.0, -14.0, -25.0, -41.0, -25.0, -14.0, -13.0, -18.0, -10.0, -6.0, -25.0, -34.

In [18]:
weather_tuples = weather_sorted_filtered.map(
    lambda row: ('&'.join((row[0][0], row[1][0])), (row[0][1], row[1][1])))  # .reduceByKey(lambda a,b:a)

print(weather_tuples.take(1))

[('01016099999&02520099999', ([4.0, 6.0, 0.0, 0.0, -53.0, -104.0, -62.0, -102.0, -72.0, -69.0, -93.0, -126.0, -86.0, -71.0, -61.0, -62.0, -27.0, -30.0, -42.0, -70.0, -49.0, -38.0, -28.0, -17.0, -15.0, -45.0, -55.0, -63.0, -19.0, -11.0, -46.0, -27.0, -41.0, -82.0, -54.0, -76.0, -114.0, -116.0, -148.0, -86.0, -35.0, -90.0, -136.0, -141.0, -84.0, -62.0, -20.0, -34.0, -53.0, -14.0, -25.0, -46.0, -19.0, -16.0, -21.0, -44.0, -39.0, -48.0, -66.0, -81.0, -88.0, -31.0, -9.0, -10.0, -15.0, -49.0, -44.0, -32.0, -36.0, -25.0, -76.0, -8.0, 1.0, -22.0, -38.0, -119.0, -136.0, -111.0, -94.0, -162.0, -122.0, -138.0, -180.0, -178.0, -122.0, -41.0, -85.0, -18.0, -15.0, -46.0, -41.0, -97.0, -159.0, -130.0, -91.0, -40.0, -44.0, -26.0, -38.0, -56.0, -70.0, -184.0, -91.0, -97.0, -99.0, -139.0, -86.0, -148.0, -162.0, -68.0, -81.0, -95.0, -84.0, -48.0, -60.0, -98.0, -95.0, -62.0, -57.0, -62.0, -43.0, -30.0, -39.0, -91.0, -103.0, -78.0, -53.0, -53.0, -14.0, -25.0, -41.0, -25.0, -14.0, -13.0, -18.0, -10.0, -6.0,

In [23]:
def avg(value_lists): return [(x[0]+x[1])/2 for x in zip(value_lists[0],value_lists[1])]
def min_x(value_lists): return [*starmap(min,zip(value_lists[0],value_lists[1]))]
def max_x(value_lists): return [*starmap(max,zip(value_lists[0],value_lists[1]))]

In [20]:
def cos_sim(value_lists):
    return sum(starmap(operator.mul, zip(value_lists[0],value_lists[1])))/\
        (sqrt(sum(starmap(operator.mul, zip(value_lists[0],value_lists[0]))))*\
            sqrt(sum(starmap(operator.mul, zip(value_lists[1],value_lists[1])))))

In [27]:
%%time 
tau=[0.90,0.90,0.90]
#Cache since it will be used thrice (for every agg function)
weather_tuples.cache()
# Cache the stocks since it will be joined with 3 seperate RDDs
stocks_sorted.cache()
#Calculate aggregated RDD's for each operation on weather data pairs
min_rdd = weather_tuples.mapValues(lambda x: min_x(x))
avg_rdd = weather_tuples.mapValues(lambda x: avg(x))
max_rdd = weather_tuples.mapValues(lambda x: max_x(x))

filtered_min = stocks_sorted.cartesian(min_rdd)
#filtered_min = filtered_min.coalesce(240)
filtered_avg = stocks_sorted.cartesian(avg_rdd)
#filtered_avg = filtered_avg.coalesce(240)
filtered_max = stocks_sorted.cartesian(max_rdd)
#filtered_max = filtered_max.coalesce(240)

# Join tables -> Format keys as (stock,loc1,loc2) and values as ([stockVals],[aggLocValues]) 
stock_loc_min = filtered_min.map(lambda row: ('&'.join((row[0][0],row[1][0])),(row[0][1],row[1][1])))
stock_loc_avg = filtered_avg.map(lambda row: ('&'.join((row[0][0],row[1][0])),(row[0][1],row[1][1])))
stock_loc_max = filtered_max.map(lambda row: ('&'.join((row[0][0],row[1][0])),(row[0][1],row[1][1])))

# Calculate cosine similarities
stock_loc_min = stock_loc_min.mapValues(cos_sim)
stock_loc_avg = stock_loc_avg.mapValues(cos_sim)
stock_loc_max = stock_loc_max.mapValues(cos_sim)

min_results = stock_loc_min.filter(lambda row: row[1]>= tau[0]).collect()
avg_results = stock_loc_avg.filter(lambda row: row[1]>= tau[1]).collect()
max_results = stock_loc_max.filter(lambda row: row[1]>= tau[2]).collect()

# Clean cache
weather_tuples.unpersist()
stocks_sorted.unpersist()

CPU times: user 103 ms, sys: 12.3 ms, total: 115 ms
Wall time: 8.16 s


In [28]:
print("Results =>")
print(f"Min Results(count={len(min_results)}): ",min_results)
print(f"Avg Results(count={len(avg_results)}): ",avg_results)
print(f"Max Results(count={len(max_results)}): ",max_results)

Results =>
Min Results(count=0):  []
Avg Results(count=4):  [('10698.Asien--Australien--Südamerika--Afrika_Japan_Tiemco-Ltd._0493AO&16153099999&40821099999', 0.9016678631487055), ('1081.Asien--Australien--Südamerika--Afrika_Australien-All-Ord_Infigen-Energy-Stap._01126U&16153099999&40821099999', 0.9113550198984206), ('10882.Asien--Australien--Südamerika--Afrika_Japan_Trans-Cosmos-Inc._049513&16153099999&40821099999', 0.9015919806464353), ('11899.Asien--Australien--Südamerika--Afrika_Singapur_Hong-Fok-Corp.-Ltd._05507B&16153099999&40821099999', 0.9002975442690063)]
Max Results(count=227):  [('10319.Asien--Australien--Südamerika--Afrika_Japan_Shinwa-Co.-Ltd._04910N&01016099999&16153099999', 0.9102297242173355), ('10698.Asien--Australien--Südamerika--Afrika_Japan_Tiemco-Ltd._0493AO&01016099999&16153099999', 0.9236036333909838), ('10319.Asien--Australien--Südamerika--Afrika_Japan_Shinwa-Co.-Ltd._04910N&02520099999&16153099999', 0.9100973030242995), ('10698.Asien--Australien--Südamerika--Af